# AI Tutor - Textbook Ingestion (Google Colab)

This notebook uses the AI Tutor's ingestion system to process textbooks.

**Features:**
- ✅ Uses TutorSystem ingestion pipeline
- ✅ Upload your own PDFs or use sample textbooks
- ✅ Creates embeddings with sentence-transformers
- ✅ Works with `chroma_data/chroma.sqlite3` database
- ✅ Save/load database from Google Drive


In [1]:
import os
from pathlib import Path


print("🔄 Cloning AI Tutor repository...")
!git clone https://github.com/HenryNVP/ai-tutor.git
%cd ai-tutor

PROJECT_ROOT = Path.cwd()
print(f"Project root: {PROJECT_ROOT}")

🔄 Cloning AI Tutor repository...
Cloning into 'ai-tutor'...
remote: Enumerating objects: 434, done.
remote: Counting objects: 100% (434/434), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 434 (delta 221), reused 406 (delta 195), pack-reused 0 (from 0)
Receiving objects: 100% (434/434), 202.27 KiB | 1.53 MiB/s, done.
Resolving deltas: 100% (221/221), done.
/content/ai-tutor
Project root: /content/ai-tutor


In [2]:
!pip install -r requirements.txt

# Add src to Python path
import sys
SRC_ROOT = PROJECT_ROOT / "src"
if str(SRC_ROOT) not in sys.path:
    sys.path.insert(0, str(SRC_ROOT))
    
print("✅ Dependencies installed")


Obtaining file:///content/ai-tutor (from -r requirements.txt (line 1))
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
# Initialize TutorSystem with chroma_data directory
from ai_tutor.system import TutorSystem
from ai_tutor.config.loader import load_settings
from pathlib import Path
import os
from google.colab import userdata

# Optional: Set API key
os.environ["OPENAI_DEFAULT_MODEL"]="gpt-4o-mini"
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

print("🚀 Loading settings...")
settings = load_settings()

# Override vector_store_dir to use chroma_data instead of data/vector_store
settings.paths.vector_store_dir = Path("chroma_data")
print(f"  Vector store directory: {settings.paths.vector_store_dir}")

# Create chroma_data directory if it doesn't exist
chroma_data_dir = PROJECT_ROOT / "chroma_data"
chroma_data_dir.mkdir(exist_ok=True)
print(f"  Created/verified chroma_data directory: {chroma_data_dir}")

print("\n🚀 Initializing TutorSystem...")
system = TutorSystem(settings=settings)

print(f"\n✓ TutorSystem initialized")
print(f"  Embedding model: {system.embedder.config.model}")
print(f"  Chunk size: {system.settings.chunking.chunk_size}")
print(f"  Vector store: {system.settings.paths.vector_store_dir}")
print(f"  Database location: {chroma_data_dir}/chroma.sqlite3")


🚀 Initializing TutorSystem...
✓ TutorSystem initialized
  Embedding model: BAAI/bge-base-en
  Chunk size: 500
  Vector store: data/vector_store


In [ ]:
# Optional: Load existing database from Google Drive
from google.colab import drive
import shutil

drive.mount('/content/drive')

# Path to database in Drive
drive_db_path = Path('/content/drive/MyDrive/ai-tutor/chroma_data/chroma.sqlite3')
local_db_path = PROJECT_ROOT / 'chroma_data' / 'chroma.sqlite3'

# Option 1: Load existing database from Drive
if drive_db_path.exists():
    print(f"📥 Loading database from Drive...")
    local_db_path.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy(drive_db_path, local_db_path)
    print(f"✅ Database loaded from Drive")
    print(f"   Location: {local_db_path}")
else:
    print(f"ℹ️  No existing database found in Drive")
    print(f"   Will create new database at: {local_db_path}")


In [13]:
from google.colab import drive

drive.mount('/content/drive')
# Data folder in Drive
drive_folder_path = '/content/drive/MyDrive/ai-tutor/raw'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
# Upload PDFs
from pathlib import Path
from google.colab import files
import shutil

raw_data_dir = PROJECT_ROOT / "data" / "raw"
raw_data_dir.mkdir(parents=True, exist_ok=True)

# Copy PDF files from Google Drive to data/raw
print(f"Copying PDF files from '{drive_folder_path}' to '{raw_data_dir}'...")
for pdf_file in Path(drive_folder_path).glob("*.pdf"):
    shutil.copy(pdf_file, raw_data_dir / pdf_file.name)
    print(f"✅ {pdf_file.name}")

# List all PDF files
pdf_files = sorted(raw_data_dir.glob("*.pdf"))
print(f"\n📚 Found {len(pdf_files)} PDF file(s) ready for ingestion:\n")
for i, pdf in enumerate(pdf_files, 1):
    size_mb = pdf.stat().st_size / (1024 * 1024)
    print(f"  {i}. {pdf.name} ({size_mb:.1f} MB)")

Copying PDF files from '/content/drive/MyDrive/ai-tutor/raw' to '/content/ai-tutor/data/raw'...
✅ collegephysicsvol1.pdf
✅ collegephysicsvol3.pdf
✅ collegephysicsvol2.pdf
✅ Algebra-and-Trigonometry-2e-WEB.pdf
✅ Calculus_Volume_1_-_WEB_68M1Z5W.pdf
✅ Principles-of-Data-Science-WEB.pdf
✅ Introduction_To_Computer_Science_-_WEB.pdf

📚 Found 7 PDF file(s) ready for ingestion:

  1. Algebra-and-Trigonometry-2e-WEB.pdf (109.6 MB)
  2. Calculus_Volume_1_-_WEB_68M1Z5W.pdf (44.5 MB)
  3. Introduction_To_Computer_Science_-_WEB.pdf (50.9 MB)
  4. Principles-of-Data-Science-WEB.pdf (32.4 MB)
  5. collegephysicsvol1.pdf (40.7 MB)
  6. collegephysicsvol2.pdf (35.2 MB)
  7. collegephysicsvol3.pdf (29.5 MB)


In [18]:
# Ingest all textbooks
# This will:
# 1. Parse PDFs
# 2. Chunk the text
# 3. Generate embeddings
# 4. Store in vector store

print("🔄 Starting ingestion (this may take a few minutes)...\n")

result = system.ingest_directory(raw_data_dir)

print("\n" + "="*60)
print("✅ INGESTION COMPLETE")
print("="*60)
print(f"📄 Documents processed: {len(result.documents)}")
print(f"📝 Chunks created: {len(result.chunks)}")
print(f"⏭️  Files skipped: {len(result.skipped)}")

if result.skipped:
    print(f"\nSkipped files:")
    for skip in result.skipped:
        print(f"  - {skip}")


🔄 Starting ingestion (this may take a few minutes)...



Ingesting documents: 100%|██████████| 7/7 [00:13<00:00,  1.94s/it]
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


✅ INGESTION COMPLETE
📄 Documents processed: 7
📝 Chunks created: 4569
⏭️  Files skipped: 0


In [19]:
# Verify ingestion
print("📊 Verification:\n")

# Check chunks file
chunks_file = PROJECT_ROOT / "data" / "processed" / "chunks.jsonl"
if chunks_file.exists():
    with open(chunks_file, 'r') as f:
        chunk_count = sum(1 for _ in f)
    print(f"✓ Chunks stored: {chunk_count}")
else:
    print("✗ No chunks file found")

# Check Chroma database
chroma_db_path = PROJECT_ROOT / "chroma_data" / "chroma.sqlite3"
if chroma_db_path.exists():
    import sqlite3
    conn = sqlite3.connect(chroma_db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    conn.close()
    print(f"✓ Chroma database exists: {chroma_db_path}")
    print(f"  Tables: {len(tables)}")
else:
    print("✗ No Chroma database found")

# Check for collection directories
chroma_data_dir = PROJECT_ROOT / "chroma_data"
collection_dirs = [d for d in chroma_data_dir.iterdir() if d.is_dir() and d.name != 'chroma_example']
if collection_dirs:
    print(f"✓ Collection directories: {len(collection_dirs)}")
else:
    print("ℹ️  No collection directories yet (will be created on first use)")

print("\n✅ Ingestion successful!")


📊 Verification:

✓ Chunks stored: 4569
✓ Embeddings: 4569 vectors of dim 768
✓ Metadata file exists

✅ Ingestion successful!


In [20]:
folders_to_download = [
    PROJECT_ROOT / "data" / "processed",
    PROJECT_ROOT / "data" / "vector_store"
]

for folder_to_download in folders_to_download:
    if folder_to_download.exists():
        zip_filename = f"{folder_to_download.name}.zip"
        # Create a zip archive of the folder
        print(f"Compressing '{folder_to_download}' into '{zip_filename}'...")
        shutil.make_archive(zip_filename.replace(".zip", ""), 'zip', folder_to_download)

        # Download the zip file
        print(f"Downloading '{zip_filename}'...")
        files.download(zip_filename)
        print(f"✅ Download initiated for {zip_filename}")
    else:
        print(f"❗️ Folder not found: {folder_to_download}. Skipping download for this folder.")

print("\nFinished processing download requests.")

Compressing '/content/ai-tutor/data/processed' into 'processed_data.zip'...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download initiated for processed_data.zip
Compressing '/content/ai-tutor/data/vector_store' into 'vector_store_data.zip'...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download initiated for vector_store_data.zip

Finished processing download requests.


In [ ]:
# Save database to Google Drive
chroma_db_path = PROJECT_ROOT / 'chroma_data' / 'chroma.sqlite3'
drive_db_dir = Path('/content/drive/MyDrive/ai-tutor/chroma_data')
drive_db_dir.mkdir(parents=True, exist_ok=True)
drive_db_path = drive_db_dir / 'chroma.sqlite3'

if chroma_db_path.exists():
    print(f"💾 Saving database to Drive...")
    shutil.copy(chroma_db_path, drive_db_path)
    size_mb = chroma_db_path.stat().st_size / (1024 * 1024)
    print(f"✅ Database saved to Drive")
    print(f"   Size: {size_mb:.2f} MB")
    print(f"   Location: {drive_db_path}")
    
    # Also save collection directories if they exist
    chroma_data_dir = PROJECT_ROOT / 'chroma_data'
    for item in chroma_data_dir.iterdir():
        if item.is_dir() and item.name != 'chroma_example':
            drive_collection_dir = drive_db_dir / item.name
            if drive_collection_dir.exists():
                shutil.rmtree(drive_collection_dir)
            shutil.copytree(item, drive_collection_dir)
            print(f"   ✅ Saved collection directory: {item.name}")
else:
    print("⚠️  No database found to save")
